1. Load microsatellite and DNAm data
2. Join
     a. create columns to join on (Case_ID, Tissue)
3. Compare data

In [1]:
import pandas as pd
import numpy as np

# read in DNAm data sample sheet
ss = pd.read_csv("../data/raw/1-4_ss.csv")
ss.head(6)

,Sample_Name,Tissue,prob_snp_outlier,prob_snp_outlier_logodds,normalized_x_intensity,normalized_y_intensity,controls_failed,detp_05,Trophoblasts,Stromal,...,Flag_cell_p_IM,PC1_mbd500,PC2_mbd500,PC3_mbd500,PC4_mbd500,PC5_mbd500,PC1_mbd500_category,PC1_mbd500_outlier,Flag_EGA_adult,horvath_EGA
0,PL20_vc,Villi,0.051761,-4.884294,0.713571,0.957759,False,False,0.352232,0.181485,...,False,-0.018943,0.024996,-0.019521,-0.048586,-0.086354,+1sd < x < +2sd,False,False,0.3544
1,PL25_chc,Chorion,0.069870,-4.470582,0.832492,0.883894,False,False,0.055959,0.171550,...,NaN,0.006405,0.085893,-0.005294,-0.012090,-0.033918,NaN,NaN,False,NaN
2,PL78_chc,Chorion,0.062602,-4.620885,0.720464,0.920460,False,False,0.000000,0.532263,...,NaN,0.001000,0.102359,0.006611,-0.060718,-0.025342,NaN,NaN,False,NaN
3,PL56_chc,Chorion,0.047357,-4.825715,1.023450,0.262161,True,False,0.200416,0.238941,...,NaN,0.000776,0.075316,-0.008978,-0.024240,-0.042781,NaN,NaN,False,NaN
4,PL15_chc,Chorion,0.029213,-5.132586,0.716309,0.926453,False,False,0.112423,0.465669,...,NaN,-0.002935,0.110974,0.014890,-0.044844,-0.033285,NaN,NaN,False,NaN
5,PL91_chc,Chorion,0.037122,-4.951654,1.008799,0.247391,False,False,0.045411,0.540959,...,NaN,-0.002607,0.097883,0.009238,-0.060132,-0.033769,NaN,NaN,False,NaN


In [2]:
# read in microsatellite data
ms_data = pd.read_excel("../data/microsatellites/2022-08-09 Contamination study result database_EA.xlsx")
ms_data.head(6)

,CASE ID,SAMPLE ID,MARKER TESTED,Marker size range,PCR CODE (note the one used in analysisONLY not the drop-outs),PCR date (yyyy-mm-dd),Analyzer Date (mm-dd-yy),ALELLE 1 size,ALELLE 2 size,ALELLE 3 size,...,ALELLE 1 area,ALELLE 2 area,ALELLE 3 area,ALELLE 4 area,Approx % contamination,result,coments,status,Unnamed: 19,Legend:
0,FT3v,FT3v,AR,~300bp,AE,2021-10-12,2021-10-13,269,272,NaN,...,UI,UI,NaN,NaN,0,Alleles too close for identification of possib...,possible female,pending; confirm other marker,NaN,Ammnion Chorion contamination study
1,NaN,FT3v,D7S3050,~150 bp,AP,2021-11-05,2021-11-09,145,149,154?,...,35293,30082,no selection,NaN,UI,one peak higher than the other and incalculabl...,NaN,"probable slight contamination; (1/3); noise, n...",NaN,NaN
2,NaN,FT3v,D7S1820,~250bp,AT,2021-11-26,2021-11-29,245,253?,262,...,95285,568,82475,NaN,0.318514,one peak higher than the other; small peak at ...,NaN,"probable slight contamination; (2/3); noise, n...",NaN,NaN
3,NaN,FT3v,D7S460,~175 bp,BO,2022-04-25,2022-04-25,177,185,NaN,...,52331,48473,NaN,NaN,0,noisy; but no evidence of contamination,NaN,no contamination,NaN,NaN
4,NaN,FT3v,D7S485,~250 bp,BR,2022-04-27,2022-04-28,241,243,NaN,...,41250,20209,NaN,NaN,0,no evidence of contamination,NaN,no contamination,NaN,NaN
5,FT22,FT22dec,AR,~300bp,AB,2021-09-23,2021-09-23,275,275,NaN,...,…,-,NaN,NaN,…,…,Overloaded Sample,pending; confirm other marker,NaN,NaN


In [3]:
#working copy of ms_data
d = ms_data

In [4]:
# these are the tissue indicators in sample name that I need to match on
d['SAMPLE ID'].replace('(\D+\d+ *)', '', regex=True).unique()

array(['v', 'dec', 'vc', ')', 'vc*', 'matwb', 'combvi', 'chc', 'amc', '',
       'comb vi', 'dec rerun', 'vc rerun', 'v ', 'vm'], dtype=object)

In [5]:
# the tissue values
ss['Tissue'].unique()

array(['Villi', 'Chorion', 'Amnion', 'Muscle', 'Spinal Cord', 'Kidney',
       'Brain', 'Basal plate', 'Cytotrophoblast', 'decidua'], dtype=object)

In [6]:
d.loc[d['SAMPLE ID'].str.contains('v'), 'Tissue'] = 'Villi' # matches v, vc, vm, vi
d.loc[d['SAMPLE ID'].str.contains('vm'), 'Tissue'] = 'Villi maternal' 
d.loc[d['SAMPLE ID'].str.contains('r2'), 'Tissue'] = 'Villi' 
d.loc[d['SAMPLE ID'].str.contains('dec'), 'Tissue'] = 'Decidua'
d.loc[d['SAMPLE ID'].str.contains('matwb'), 'Tissue'] = 'Maternal blood'
d.loc[d['SAMPLE ID'].str.contains('dec'), 'Tissue'] = 'decidua'
d.loc[d['SAMPLE ID'].str.contains('ch'), 'Tissue'] = 'Chorion'
d.loc[d['SAMPLE ID'].str.contains('am'), 'Tissue'] = 'Amnion'
print(d[['SAMPLE ID', 'Tissue']].sort_values('Tissue').to_string())

            SAMPLE ID          Tissue
277          PL56 am3          Amnion
235          PL31 am3          Amnion
227          PL31 amc          Amnion
223          PL31 amc          Amnion
306          PL91 am3          Amnion
233          PL31 am1          Amnion
303          PL91 am3          Amnion
302          PL91 am2          Amnion
301          PL91 am1          Amnion
298          PL91 am3          Amnion
297          PL91 am2          Amnion
296          PL91 am1          Amnion
267          PL56 am1          Amnion
234          PL31 am2          Amnion
268          PL56 am2          Amnion
269          PL56 am3          Amnion
274          PL56 am3          Amnion
272          PL56 am1          Amnion
273          PL56 am2          Amnion
232          PL31 ch3         Chorion
231          PL31 ch2         Chorion
230          PL31 ch1         Chorion
226          PL31 chc         Chorion
222          PL31 chc         Chorion
206        NTD14matwb  Maternal blood
185         

In [7]:
# Extract case ID
d['Case_ID'] = d['SAMPLE ID'].str.extract(pat = '(\D+\d+)')

# Inspect result
print(
    d[['SAMPLE ID', 'Case_ID', 'Tissue', 'Approx % contamination']].sort_values(['Tissue', 'Case_ID'])
    .to_string() # see the whole dataframe
)

            SAMPLE ID Case_ID          Tissue Approx % contamination
223          PL31 amc    PL31          Amnion              27.239418
227          PL31 amc    PL31          Amnion              39.176876
233          PL31 am1    PL31          Amnion               26.55483
234          PL31 am2    PL31          Amnion              23.400351
235          PL31 am3    PL31          Amnion              34.269964
267          PL56 am1    PL56          Amnion                      0
268          PL56 am2    PL56          Amnion                      0
269          PL56 am3    PL56          Amnion                     ??
272          PL56 am1    PL56          Amnion                      0
273          PL56 am2    PL56          Amnion                      0
274          PL56 am3    PL56          Amnion                     ??
277          PL56 am3    PL56          Amnion                     ??
296          PL91 am1    PL91          Amnion                      0
297          PL91 am2    PL91     

In [8]:
# Summarize % contamination: step 1 convert to numeric
d['contamination_percent'] = pd.to_numeric(
    d['Approx % contamination'], 
    errors = 'coerce' # converts letters into NaN
)

print(
    d[['SAMPLE ID', 'Case_ID', 'Tissue', 'Approx % contamination', 'contamination_percent']].sort_values(['Tissue', 'Case_ID'])
    .to_string() # see the whole dataframe
)

            SAMPLE ID Case_ID          Tissue Approx % contamination  contamination_percent
223          PL31 amc    PL31          Amnion              27.239418              27.239418
227          PL31 amc    PL31          Amnion              39.176876              39.176876
233          PL31 am1    PL31          Amnion               26.55483              26.554830
234          PL31 am2    PL31          Amnion              23.400351              23.400351
235          PL31 am3    PL31          Amnion              34.269964              34.269964
267          PL56 am1    PL56          Amnion                      0               0.000000
268          PL56 am2    PL56          Amnion                      0               0.000000
269          PL56 am3    PL56          Amnion                     ??                    NaN
272          PL56 am1    PL56          Amnion                      0               0.000000
273          PL56 am2    PL56          Amnion                      0            

In [9]:
# Summarize
d_sum = (d
         .dropna(subset = ['contamination_percent'])
         .groupby(['Case_ID', 'Tissue'])
         .agg(
             mean_contam=pd.NamedAgg(column = 'contamination_percent', aggfunc='mean'),
             count = pd.NamedAgg(column = 'contamination_percent', aggfunc = 'size'),
             min = pd.NamedAgg(column = 'contamination_percent', aggfunc = 'min'),
             max = pd.NamedAgg(column = 'contamination_percent', aggfunc = 'max')
         )
)

print(d_sum)

                        mean_contam  count        min        max
Case_ID Tissue                                                  
FT22    Villi              3.290757      2   2.355604   4.225910
FT26    Villi              0.382575      2   0.000000   0.765150
FT28    Villi              1.784999      1   1.784999   1.784999
FT3     Villi              0.079629      4   0.000000   0.318514
FT39    Villi              1.961109      3   1.058070   2.424091
FT41    Villi              1.837805      3   1.204407   2.740107
FT42    Villi              0.741450      2   0.734145   0.748754
FT59    Villi              0.149383      2   0.000000   0.298766
FT60    Villi              1.484404      2   1.249239   1.719568
FT73    Villi              1.232453      1   1.232453   1.232453
FT75    Villi              0.000000      1   0.000000   0.000000
FT78    Villi              0.000000      2   0.000000   0.000000
NTD14   Villi              0.000000      2   0.000000   0.000000
NTD3    Villi            

In [10]:
d.dropna(subset = ['contamination_percent'])

,CASE ID,SAMPLE ID,MARKER TESTED,Marker size range,PCR CODE (note the one used in analysisONLY not the drop-outs),PCR date (yyyy-mm-dd),Analyzer Date (mm-dd-yy),ALELLE 1 size,ALELLE 2 size,ALELLE 3 size,...,ALELLE 4 area,Approx % contamination,result,coments,status,Unnamed: 19,Legend:,Tissue,Case_ID,contamination_percent
0,FT3v,FT3v,AR,~300bp,AE,2021-10-12,2021-10-13,269,272,NaN,...,NaN,0,Alleles too close for identification of possib...,possible female,pending; confirm other marker,NaN,Ammnion Chorion contamination study,Villi,FT3,0.000000
2,NaN,FT3v,D7S1820,~250bp,AT,2021-11-26,2021-11-29,245,253?,262,...,NaN,0.318514,one peak higher than the other; small peak at ...,NaN,"probable slight contamination; (2/3); noise, n...",NaN,NaN,Villi,FT3,0.318514
3,NaN,FT3v,D7S460,~175 bp,BO,2022-04-25,2022-04-25,177,185,NaN,...,NaN,0,noisy; but no evidence of contamination,NaN,no contamination,NaN,NaN,Villi,FT3,0.000000
4,NaN,FT3v,D7S485,~250 bp,BR,2022-04-27,2022-04-28,241,243,NaN,...,NaN,0,no evidence of contamination,NaN,no contamination,NaN,NaN,Villi,FT3,0.000000
8,NaN,FT22v,D7S1820,~250bp,AI,2021-10-20,2021-10-27,250,250,254,...,NaN,2.355604,Slight contamination with maternal dec in villi,"homozygous, contamination alleles overlap; can...",NaN,NaN,NaN,Villi,FT22,2.355604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,NaN,PM372 vm,AR,~300bp,AY,2022-01-26,2022-01-27,259,273,NaN,...,NaN,19.386253,heavy maternal contamination,possible male,NaN,NaN,NaN,Villi maternal,PM372,19.386253
424,NaN,PM375 vc,AR,~300bp,AY,2022-01-26,2022-01-27,270,282,NaN,...,NaN,0,female,NaN,NaN,NaN,NaN,Villi,PM375,0.000000
425,NaN,PM375 vm,AR,~300bp,AY,2022-01-26,2022-01-27,270,282,285,...,NaN,5.422752,heavy maternal contamination; maternal 285 α- ...,NaN,NaN,NaN,NaN,Villi maternal,PM375,5.422752
427,NaN,PM376 vc,AR,~300bp,AY,2022-01-26,2022-01-27,267,276,NaN,...,NaN,0,female,NaN,NaN,NaN,NaN,Villi,PM376,0.000000


# Compare to the list of samples I originally sent
to get validated with microsatellites
goal is to account for all samples

In [11]:
follow_up =  pd.read_csv("../data/raw/1-6_DNAm contamination Victor follow-up samples.csv")
follow_up

,Sample_Name,dataset,dataset_label,Sex
0,PL18_vc,1,Konwar 2018,F
1,FT28_vc,4,Price 2016,F
2,FT75_vc,4,Price 2016,F
3,FT73_v,4,Price 2016,F
4,FT27_v,4,Price 2016,M
5,PL118_vc,4,Price 2016,M
6,FT26_v,4,Price 2016,M
7,FT74_vc,4,Price 2016,M
8,FT22_v,4,Price 2016,M
9,PL94_vc,1,Konwar 2018,M


In [12]:
# get case ID and tissue from sample name
follow_up['Case_ID'] = follow_up['Sample_Name'].str.extract(pat = '(\D+\d+)')
follow_up['Tissue'] = 'Villi'

print(follow_up[['Sample_Name', 'Case_ID', 'Tissue', 'dataset']].sort_values('dataset').to_string())

   Sample_Name Case_ID Tissue  dataset
0      PL18_vc    PL18  Villi        1
26    PL123_vc   PL123  Villi        1
9      PL94_vc    PL94  Villi        1
17     PL57_vc    PL57  Villi        1
12     PL34_vc    PL34  Villi        1
13     PL56_vc    PL56  Villi        1
16    PL112_vc   PL112  Villi        1
25    PL96v1v2    PL96  Villi        2
22     PL64_r2    PL64  Villi        2
24      FT39_v    FT39  Villi        4
23     NTD_9_v   NTD_9  Villi        4
21      FT41_v    FT41  Villi        4
20     FT78_vc    FT78  Villi        4
19       FT3_v     FT3  Villi        4
18     NTD_3_v   NTD_3  Villi        4
14      FT59_v    FT59  Villi        4
27      FT42_v    FT42  Villi        4
11      FT60_v    FT60  Villi        4
10    PL148_vc   PL148  Villi        4
8       FT22_v    FT22  Villi        4
7      FT74_vc    FT74  Villi        4
6       FT26_v    FT26  Villi        4
5     PL118_vc   PL118  Villi        4
4       FT27_v    FT27  Villi        4
3       FT73_v    FT73  V

In [13]:
#join
follow_up[
    # returns indices where follow up Case_ID is not in d_sum
    np.logical_not(follow_up
    .Case_ID
    .isin(d_sum.reset_index().Case_ID)
                  )
]

,Sample_Name,dataset,dataset_label,Sex,Case_ID,Tissue
4,FT27_v,4,Price 2016,M,FT27,Villi
7,FT74_vc,4,Price 2016,M,FT74,Villi
12,PL34_vc,1,Konwar 2018,F,PL34,Villi
15,FT38_v,4,Price 2016,M,FT38,Villi
18,NTD_3_v,4,Price 2016,M,NTD_3,Villi
23,NTD_9_v,4,Price 2016,M,NTD_9,Villi


Why aren't these samples in the microsatellites results file? Manually inspect.

Looks like the FT27,74,38 were uninformative so they were removed in the data processing
NTDs case ID are still not matching (underscore needs removal)
PL is UI still

FT_27_v says UI but notes suggest that there was no evidence of 3rd allele, therefore should be 0.
FT74 uninformative
FT38 looks uninformative or likely 0

NTDD3 is informative, reason for not matching is case ID is NTD3 not NTD_3
NTD9 same as above

----
Evie Aug 11:
FT27v no decidua sample. Likely no maternal contamination because no 3rd allele detected in 4 markers. 
FT74 - Retrospectively looks like no contamination
FT38 difficult to discern because overlapping peaks. keep UI

---
Conclusion: 

FT27v set to 0 

FT74v set to 0

FT38 keep UI 

PL34 keep UI

NTD3/9 make sure SS case_ID is fixed

In [14]:
d_sum.reset_index().Case_ID

0      FT22
1      FT26
2      FT28
3       FT3
4      FT39
5      FT41
6      FT42
7      FT59
8      FT60
9      FT73
10     FT75
11     FT78
12    NTD14
13     NTD3
14     NTD9
15    PL112
16    PL118
17    PL123
18    PL148
19     PL18
20    PL203
21    PL204
22    PL215
23    PL218
24    PL277
25    PL278
26     PL31
27     PL31
28     PL31
29     PL56
30     PL56
31     PL57
32     PL64
33     PL91
34     PL91
35     PL94
36     PL96
37    PM372
38    PM372
39    PM375
40    PM375
41    PM376
42    PM376
Name: Case_ID, dtype: object

In [53]:
#working copy of data
ss_w = ss

#get case id in ss_w
ss_w['Case_ID'] = ss_w['Sample_Name'].str.extract(pat = '(\D+\d+)')

# how many of d_sum are in ss
print(str(sum(d_sum.reset_index().Case_ID.isin(ss_w['Case_ID']))) + '/' + str(len(d_sum)))

29/43


In [54]:
# check mismatches
d_sum.reset_index()[
    ~d_sum.reset_index().Case_ID.isin(ss_w['Case_ID'])
]


,Case_ID,Tissue,mean_contam,count,min,max
13,NTD3,Villi,1.332443,5,1.008325,1.667836
14,NTD9,Villi,4.480082,2,3.238896,5.721268
20,PL203,Villi,0.000000,1,0.000000,0.000000
21,PL204,Villi,0.000000,1,0.000000,0.000000
22,PL215,Villi,14.691497,1,14.691497,14.691497
23,PL218,Villi,1.357834,1,1.357834,1.357834
24,PL277,Villi,0.000000,1,0.000000,0.000000
25,PL278,Villi,0.000000,3,0.000000,0.000000
37,PM372,Villi,0.000000,1,0.000000,0.000000
38,PM372,Villi maternal,19.386253,1,19.386253,19.386253


In [55]:
# check NTDs and PLs. The PMs are additional samples not originally included in the DNAm project
ss_w[ss_w['Sample_Name'].str.contains('NTD')].head()

,Sample_Name,Tissue,prob_snp_outlier,prob_snp_outlier_logodds,normalized_x_intensity,normalized_y_intensity,controls_failed,detp_05,Trophoblasts,Stromal,...,PC1_mbd500,PC2_mbd500,PC3_mbd500,PC4_mbd500,PC5_mbd500,PC1_mbd500_category,PC1_mbd500_outlier,Flag_EGA_adult,horvath_EGA,Case_ID
260,NTD14_vc,Villi,0.125696,-4.043148,0.732825,0.873294,False,False,0.415371,0.247990,...,-0.018719,0.043129,0.007061,-0.055314,0.003999,+1sd < x < +2sd,False,False,0.411255,NTD14
266,NTD_1_v,Villi,0.027448,-5.332511,1.050086,0.105300,False,False,0.329481,0.207932,...,-0.022156,0.025635,0.003971,-0.042564,0.009058,mean < x < +1sd,False,False,0.134849,NTD_1
332,NTD_8_mus,Muscle,0.040294,-5.209862,0.737632,0.867233,False,False,0.088946,0.303161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NTD_8
335,NTD_3_mus,Muscle,0.024643,-5.434555,0.735252,0.866257,False,False,0.084840,0.320717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NTD_3
337,NTD6_sc,Spinal Cord,0.084220,-4.482780,0.719578,0.873525,False,False,0.080035,0.212430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NTD6


In [59]:
# remove '_'
ss_w['Case_ID'] = ss_w['Case_ID'].str.replace('(NTD)_(\d).*', '\\1\\2', regex = True)

#print result
ss_w[['Sample_Name', 'Case_ID']][ss_w['Case_ID'].str.contains('NTD')]

,Sample_Name,Case_ID
260,NTD14_vc,NTD14
266,NTD_1_v,NTD1
332,NTD_8_mus,NTD8
335,NTD_3_mus,NTD3
337,NTD6_sc,NTD6
341,NTD_8_sc,NTD8
342,NTD16_kid,NTD16
343,NTD_9_sc,NTD9
348,NTD17_sc,NTD17
356,NTD_8_kid,NTD8


In [60]:
# how many of d_sum are in ss
print(str(sum(d_sum.reset_index().Case_ID.isin(ss_w['Case_ID']))) + '/' + str(len(d_sum)))

31/43


In [63]:
ss_w[ss_w['Case_ID'].str.contains('PL2')][['Sample_Name', 'Case_ID']]

,Sample_Name,Case_ID
0,PL20_vc,PL20
1,PL25_chc,PL25
12,PL25_amc,PL25
33,PL26_vc,PL26
34,PL25_vc,PL25
45,PL20_chc,PL20
50,PL20_amc,PL20
57,PL28_vc,PL28
58,PL23_vc,PL23
68,PL29_vc,PL29


In [16]:
#working copy of data
ss_w = ss

for i in ss_w.columns:
    print(i)

Sample_Name
Tissue
prob_snp_outlier
prob_snp_outlier_logodds
normalized_x_intensity
normalized_y_intensity
controls_failed
detp_05
Trophoblasts
Stromal
Hofbauer
Endothelial
nRBC
Syncytiotrophoblast
GA_epi_RPC
GA_epi_CPC
GA_epi_RRPC
GA
Sex
Group
dataset_label
dataset_label_short
GEO_accession
Tissue_gen
Tissue_gen2
GA_cat
Outlier
dataset
Flag_XY
Flag_XY_contam
GA_cat_combined
GA_cat_epi
Flag_cell_p_TB
Flag_cell_p_IM
PC1_mbd500
PC2_mbd500
PC3_mbd500
PC4_mbd500
PC5_mbd500
PC1_mbd500_category
PC1_mbd500_outlier
Flag_EGA_adult
horvath_EGA


In [17]:
ss_w.dataset_label_short.unique()

array(['ACA', 'PE1', 'PE2', 'NTD', 'ART', nan, 'RSA'], dtype=object)

In [18]:
# Compare

In [19]:
name = "Mark"
age = 31
message = f"Hello {name} ({age})"
print(message)

Hello Mark (31)
